# AI PPTX Creator

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-OpenAI-API-KEY" data-toc-modified-id="1.-OpenAI-API-KEY-1">1. OpenAI API KEY</a></span></li><li><span><a href="#2.-Testing-GPT4-from-LangChain" data-toc-modified-id="2.-Testing-GPT4-from-LangChain-2">2. Testing GPT4 from LangChain</a></span></li><li><span><a href="#3.-Loading-PDF-file" data-toc-modified-id="3.-Loading-PDF-file-3">3. Loading PDF file</a></span></li><li><span><a href="#4.-Embedding-Model-and-vector-database" data-toc-modified-id="4.-Embedding-Model-and-vector-database-4">4. Embedding Model and vector database</a></span></li></ul></div>

## 1. OpenAI API KEY
To carry out this project, we will need an API KEY from OpenAI to use the GPT-4 Turbo model. This API KEY can be obtained at https://platform.openai.com/api-keys. It is only displayed once, so it must be saved at the moment it is obtained. Of course, we will need to create an account to get it.

We store the API KEY in a .env file to load it with the dotenv library and use it as an environment variable. This file is added to the .gitignore to ensure that it cannot be seen if we upload the code to GitHub, for example.

In [1]:
# import API KEY

import os                           # operating system library
from dotenv import load_dotenv      # load environment variables  


load_dotenv()


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## 2. Testing GPT4 from LangChain

We are going to test the connection from LangChain to the GPT-4 model. Let's ask what the Suez canal is.

In [2]:
from langchain_openai.chat_models import ChatOpenAI   # LangChain connection to OpenAI

model = ChatOpenAI(model="gpt-4-turbo")

response = model.invoke("What is the Suez Canal?")

response.content

"The Suez Canal is a man-made waterway in Egypt, connecting the Mediterranean Sea to the Red Sea. It was officially opened in 1869 and is one of the world's most heavily used shipping lanes. The canal provides a direct route for maritime traffic between Europe and Asia, bypassing the need to sail around Africa through the Cape of Good Hope. This significantly reduces the travel distance and time for ships, making it a critical route for international trade.\n\nThe canal is approximately 193 kilometers (120 miles) long, and its construction was a monumental engineering feat led by the French engineer Ferdinand de Lesseps, although it was built largely by forced Egyptian labor. The Suez Canal has no locks, as the sea level is the same at both ends, and it can accommodate ships of various sizes, including some of the largest vessels, known as supertankers.\n\nOver the years, the canal has been expanded and modernized to accommodate increasing maritime traffic and larger ships. It is also 

## 3. Loading PDF file

Now, we load the [Recent threats in the Red Sea](https://www.europarl.europa.eu/RegData/etudes/BRIE/2024/760390/EPRS_BRI(2024)760390_EN.pdf) PDF file previously downloaded, a briefing published by the European Parliament in 2024.

In [3]:
os.listdir("../pdfs")

['.DS_Store', '2024_BRIEFING.pdf']

In [4]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [5]:
loader = PyPDFDirectoryLoader("../pdfs/")

pages = loader.load()

In [6]:
len(pages)

8

## 4. Embedding Model and vector database

Embeddings transform data, especially textual data, into a format, usually a vector of numbers, that ML algorithms can process effectively. These embeddings capture the contextual relationships and semantic meanings of words, phrases, or documents, enabling various applications in AI.

Chroma DB is an open-source vector database designed to store and retrieve vector embeddings efficiently. It is particularly useful for enhancing LLMs by providing relevant context to user inquiries. Chroma DB allows for the storage of embeddings along with metadata, which can later be utilized by LLMs or for semantic search engines over text data.

In [7]:
from langchain_openai.embeddings import OpenAIEmbeddings


vectorizer = OpenAIEmbeddings()

In [8]:
from langchain_community.vectorstores import Chroma

chroma_db = Chroma.from_documents(pages, vectorizer, persist_directory="../chroma_db")

In [9]:
retriever = chroma_db.as_retriever(search_type="mmr", search_kwargs={"k": 2, "lambda_mult": 0.25})

In [10]:
from langchain.prompts import ChatPromptTemplate

In [11]:
template = """
            Given the context below and the question, 
            please generate a header and 10 bullet points.
            List with numbers the bullet points.
            Summarize each bullet point in 40 words.
            
            Put a line separator after `:` symbol.

            Context: {context}

            Question: {question}
            """


prompt = ChatPromptTemplate.from_template(template)

In [12]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [13]:
from langchain_core.runnables import RunnablePassthrough

In [14]:
query = "What are the endnotes of the briefing?"

In [15]:
in_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model | parser


response = in_chain.invoke(query)

In [16]:
response.split("\n")

['**Header: Endnotes of the March 2024 EPRS Briefing on Recent Threats in the Red Sea**',
 '',
 '1. **Authorship and Research Team**:',
 '   - The briefing was authored by Angelos Delivorias with graphic support from Samy Chahri, under the European Parliamentary Research Service.',
 '',
 '2. **Document Identification**:',
 '   - The document is cataloged as PE 760.390 and was published in March 2024, labeled in English (EN).',
 '',
 '3. **Main Subject Focus**:',
 '   - The document focuses on the recent attacks by the Iran-backed Houthi militia on commercial shipping in the Red Sea, particularly near the Bab el-Mandeb Strait.',
 '',
 '4. **Economic Repercussions Addressed**:',
 '   - It discusses the economic impacts on regional and EU economies, highlighting disruptions in global supply chains and potential inflationary pressures.',
 '',
 '5. **Shipping Route Alternatives**:',
 '   - Due to the attacks, major shipping companies have suspended transits through the Suez Canal, opting fo

In [17]:
from langchain_openai import OpenAI

input_model = OpenAI(temperature=0, max_tokens=1024)

In [18]:
template = """
            We have provided  information below.
            Given this information, please generate a python code with python-pptx for three 
            slide presentation with this information. 
            
            Put the title in the first slide, 
            5 bullet points in the second slide and another 5 bullet in the third slide.
            Put list number in each bullet point.
                        
            Separate the bullet points into separate texts with line separator.
            Set font size to 20 for each bullet point. 
            Save the file in ../pptx/Red Sea Security Threats.pptx path

            Information: {context}
            """


prompt = ChatPromptTemplate.from_template(template)

In [19]:
out_chain = prompt | input_model | parser

In [20]:
output = out_chain.invoke({"context": response})

In [21]:
output.split("\n")

['',
 '# Import necessary libraries',
 'from pptx import Presentation',
 'from pptx.util import Pt',
 '',
 '# Create new presentation',
 'prs = Presentation()',
 '',
 '# Add title slide',
 'slide = prs.slides.add_slide(prs.slide_layouts[0])',
 'title = slide.shapes.title',
 'title.text = "Endnotes of the March 2024 EPRS Briefing on Recent Threats in the Red Sea"',
 '',
 '# Add second slide with 5 bullet points',
 'slide = prs.slides.add_slide(prs.slide_layouts[1])',
 'title = slide.shapes.title',
 'title.text = "Authorship and Research Team"',
 '',
 '# Add bullet points',
 'bullet_points = slide.shapes.placeholders[1].text_frame',
 'bullet_points.text = "1. The briefing was authored by Angelos Delivorias with graphic support from Samy Chahri, under the European Parliamentary Research Service.\\n2. The document is cataloged as PE 760.390 and was published in March 2024, labeled in English (EN).\\n3. The document focuses on the recent attacks by the Iran-backed Houthi militia on commerci

In [22]:
exec("print(2+2)")

4


In [23]:
exec(output)